# Data parsing

In [ ]:
# Importing Libraries
import pandas as pd
import re

In [ ]:
# Function to parse the files
def parse_ASCII_file(line):
  # Skip empty lines or without relevant data
  if not line.strip() or 'Begin Packet' in line:
    return None

  # Extract columns using regex
  columns={'Type':r'type=(\d+)',
           'ts':r'ts=(\d+)',
           'txTs':r'txTs=(\d+)',
           'OrderId':r'orderId=(\d+)',
           'Price':r'price=(\d+)',
           'Quantity':r'qty=(\d+)',
           'Trader':r'trader=(\d+)',
           'Symbol':r'symbol=(\d+)',
           'IsBuy':r'isBUY=(\d+)'}

  # Initialize row
  row={}
  for key,pattern in columns.items():
    match=re.search(pattern,line)
    row[key]=int(match.group(1)) if match else None

  # Extract Book info if present
  book_match=re.search(r'Book: \[Bid: (\d+) @ px (\d+) \((\d+) orders\) \| Ask: qty(\d+) @ px (\d+) \((\d+) orders\)\]', line)
  if book_match:
    row['book_qty']=int(book_match.group(1))
    row['bid_price']=int(book_match.group(2))
    row['bid_orders']=int(book_match.group(3))
    row['ask_qty']=int(book_match.group(4))
    row['ask_price']=int(book_match.group(5))
    row['ask_orders']=int(book_match.group(6))
  else:
    row['bid_qty']=row['bid_price']=row['bid_orders']=None
    row['ask_qty']=row['ask_price']=row['ask_orders']=None

  return row

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing the work directory
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
# Read and parse file lines. Starting with Rainbow file.
with open ('JumpTradingInterview-MarketData-ASCII.RAINBOW.txt', 'r', encoding='ASCII') as f:
  lines=f.readlines()

# Parse lines into a list of dictionaries  and remove none entries
data=[parse_ASCII_file(line) for line in lines]
data=[d for d in data if d is not None]

# Create a DataFrame
market_data_df=pd.DataFrame(data)

# Filtering for only add events for now
market_data_df=market_data_df[market_data_df['type']==1]

# Saving to the location
market_data_df.to_csv('JumpTradingInterview-MD_RAINBOW.csv', index=False)


In [ ]:
# Doing the same for Unicorn file.
with open ('JumpTradingInterview-MarketData-ASCII.UNICORN.txt', 'r', encoding='ASCII') as f:
  lines=f.readlines()

# Parse lines into a list of dictionaries  and remove none entries
data=[parse_ASCII_file(line) for line in lines]
data=[d for d in data if d is not None]

# Create a DataFrame
market_data_df=pd.DataFrame(data)

# Filtering for only add events for now
market_data_df=market_data_df[market_data_df['type']==1]

# Saving to the location
market_data_df.to_csv('JumpTradingInterview-MD_UNICORN.csv', index=False)


# Data Prep and Combining the 2 Files

In [ ]:
# Load the 2 files and Concatenate
md_rainbow_df=pd.read_csv('JumpTradingInterview-MD_RAINBOW.csv')
md_unicorn_df=pd.read_csv('JumpTradingInterview-MD_UNICORN.csv')

market_data_df=pd.DataFrame() #Reassigning it as an empty DataFrame
market_data_df=pd.concat([md_rainbow_df, md_unicorn_df])

In [ ]:
# convert time format to relevant timestamp to match the fill report files
market_data_df['ts']=pd.to_datetime(market_data_df['ts'], unit='ns') # Using the same unit as in for the Fill reports
market_data_df['txTs']=pd.to_datetime(market_data_df['txTs'], unit='ns')

In [ ]:
# Map Symbols
'''
These files use numbers to deonote the symbols while the fill files use names.
Mapping the numbers to names on the basis of followign logic.
Since RAINBOWZ0 is more active, so whichever is more will be assigned that and other will be assigned
UNICORNZ0.
'''
market_data_df['symbol']=market_data_df['symbol'].map({1:'RAINBOWZ0', 2:'UNICORNZ0'})

In [ ]:
# Saving it into a new CSV
market_data_df.to_csv('MD-Combined.csv', index=False)